# Find Spatial Locations
# Buffer a point, return all data within spatial geometry of buffer

In [1]:
# import libraries
import getpass
from arcgis.gis import *
from arcgis import geometry
from arcgis import features


In [2]:
# Utilize getpass for password management in Jupyter notebooks
password = getpass.getpass()

········


In [3]:
# Connect to GIS
gis = GIS('http://www.arcgis.com', 'abrown_citygov', password)
gis

GIS @ http://citygov.maps.arcgis.com version:6.4

In [4]:
# Manually create a point
pt_coords = features.feature.Point({"x" : -75.127341, "y" : 39.981295, "spatialReference" : {"wkid" : 4326}})
pt = geometry.Point(pt_coords)
feat = features.Feature(geometry=pt, attributes={'OBJECTID': 1,
                                                        'name': 'Test_point',
                                                        'type': 'Feature'})

# Convert the feature into a feature set to utilize in geoprocessing tools
fset = features.FeatureSet([feat])
fset

<FeatureSet> 1 features

In [5]:
# Search for Philadelphia Redacted Crime, put query directly in search
crime_s = gis.content.search("title:PhillyCrime", item_type="Feature Service")
incidents_service = crime_s[0]
incident_layers = incidents_service.layers
incidents = incident_layers[0]

In [50]:
# Create a map object, add layers then visualize
map_1 = gis.map("2143 York Street, Philadelphia, PA 19125")
map_1.add_layer(incidents)
map_1.add_layer(fset)
map_1

MapView(layout=Layout(height='400px', width='100%'))

In [15]:
# Buffer the point
point_buff = features.use_proximity.create_buffers(fset.to_dict(), distances=[0.1], units = 'Miles')
map_1.add_layer(point_buff)

In [21]:
# Run summarize within tool to look at how many points fall within the boundary.
output = features.analysis.summarize_within(sum_within_layer=point_buff, summary_layer=incidents, sum_shape=True, 
                                   summary_fields=[], group_by_field='text_gener', minority_majority=False, 
                                   percent_shape=False, output_name=None, context=None, gis=None)

In [65]:
# The output of the summarize within did not specifify an output_name, the result is a dictionary.  Grab the dictionary key
# based on "result_layer"
print(output)
print(type(output))
# Return just FeatureCollection
final = output.get('result_layer')

{'result_layer': <FeatureCollection>, 'group_by_summary': <FeatureCollection>}
<class 'dict'>


In [48]:
# FeatureCollections are different than FeatureLayerCollections; unless one saves the layer using output_name
# a where clause cannot be utilized.
finallayers = final.query()
finallayers

<FeatureSet> 1 features

In [49]:
# Show query results in data table
finallayers.sdf

,AnalysisArea,BUFF_DIST,Join_ID,OBJECTID,ORIG_FID,Point_Count,SHAPE,name,type
0,0.081325,0.1,1,1,1,228,"{""rings"": [[[-75.12734031650234, 39.9827442170...",Test_point,Feature


In [61]:
# Create a second map object for clarity
map_2 = gis.map("2143 York Street, Philadelphia, PA 19125")
map_2.add_layer(finallayers)
map_2

MapView(layout=Layout(height='400px', width='100%'))